In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math

In [169]:
amazon_del = pd.read_csv("amazon_delivery.csv")
amazon_del.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43739 entries, 0 to 43738
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order_ID         43739 non-null  object 
 1   Agent_Age        43739 non-null  int64  
 2   Agent_Rating     43685 non-null  float64
 3   Store_Latitude   43739 non-null  float64
 4   Store_Longitude  43739 non-null  float64
 5   Drop_Latitude    43739 non-null  float64
 6   Drop_Longitude   43739 non-null  float64
 7   Order_Date       43739 non-null  object 
 8   Order_Time       43739 non-null  object 
 9   Pickup_Time      43739 non-null  object 
 10  Weather          43648 non-null  object 
 11  Traffic          43739 non-null  object 
 12  Vehicle          43739 non-null  object 
 13  Area             43739 non-null  object 
 14  Delivery_Time    43739 non-null  int64  
 15  Category         43739 non-null  object 
dtypes: float64(5), int64(2), object(9)
memory usage: 5.3+ MB


# **Introduction**  


## **Quick look column by column**  

### **Order_ID**  
Should be equivalent to PRIMARY KEY of a SQL table, only unique values. If so it can be removed when building a machine learning.  

In [170]:
duplicates_order = amazon_del.duplicated(subset=["Order_ID"])
print(len(amazon_del[duplicates_order]))

0


All IDs are different.

### **Agent_Age**  
On first sight everything seems ok. Type: integer and no missing values.  


In [171]:
amazon_del["Agent_Age"].describe()

count    43739.000000
mean        29.567137
std          5.815155
min         15.000000
25%         25.000000
50%         30.000000
75%         35.000000
max         50.000000
Name: Agent_Age, dtype: float64

No Age = 0 or other abeerrant numbers.

### **Agent Rating**  
Based on the info() table there are apparent missing values (54).

In [172]:
missing_ratings = amazon_del[amazon_del["Agent_Rating"].isna()]
non_missing_ratings = amazon_del[~(amazon_del["Agent_Rating"].isna())]
len(missing_ratings)/len(amazon_del)

0.0012345961270262237

I could potentially remove the values as they represent less than 1%. But I will see if other values are missing.

In [173]:
missing_ratings["Traffic"].value_counts(dropna=False)

Traffic
Low        23
Jam        15
Medium     14
High        2
Name: count, dtype: int64

In [174]:
missing_ratings["Weather"].value_counts(dropna=False)

Weather
Windy         12
Sunny         11
Cloudy        11
Stormy         8
Sandstorms     7
Fog            5
Name: count, dtype: int64

I can remove them. There are no obvious reason of these missing values.  

In [175]:
amazon_del = amazon_del.drop(missing_ratings.index, axis=0)
amazon_del.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43685 entries, 0 to 43738
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order_ID         43685 non-null  object 
 1   Agent_Age        43685 non-null  int64  
 2   Agent_Rating     43685 non-null  float64
 3   Store_Latitude   43685 non-null  float64
 4   Store_Longitude  43685 non-null  float64
 5   Drop_Latitude    43685 non-null  float64
 6   Drop_Longitude   43685 non-null  float64
 7   Order_Date       43685 non-null  object 
 8   Order_Time       43685 non-null  object 
 9   Pickup_Time      43685 non-null  object 
 10  Weather          43594 non-null  object 
 11  Traffic          43685 non-null  object 
 12  Vehicle          43685 non-null  object 
 13  Area             43685 non-null  object 
 14  Delivery_Time    43685 non-null  int64  
 15  Category         43685 non-null  object 
dtypes: float64(5), int64(2), object(9)
memory usage: 5.7+ MB


### **Store Latitude**  
When the latitude is positive we are above the equator line. When the latitude is negative we are south to the equator line.  

In [176]:
amazon_del["Store_Latitude"].describe()

count    43685.000000
mean        17.214543
std          7.750885
min        -30.902872
25%         12.933298
50%         18.551440
75%         22.732225
max         30.914057
Name: Store_Latitude, dtype: float64

### **Store Longitude**  
No apparent missing values.

In [177]:
amazon_del["Store_Longitude"].describe()

count    43685.000000
mean        70.668593
std         21.459258
min        -88.366217
25%         73.170283
50%         75.898497
75%         78.045359
max         88.433452
Name: Store_Longitude, dtype: float64

In [178]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [179]:
from geopy.geocoders import Nominatim
from modules_amazon import Location

#geolocator = Nominatim(user_agent="my_loc_app")
#def country(Latitude,Longitude):
#    location = geolocator.reverse(Latitude+','+Longitude)
#    address = location.raw["address"]
#    country = address.get("country","")
#    return country
latitude_0 = amazon_del.iloc[0,3]
longitude_0 = amazon_del.iloc[0,4]
loc = Location(latitude_0,longitude_0)
print(loc.city())
print(loc.country())


Indore
India


### **Order_Date**  
No apparent missing values. I will switch to date format.

In [180]:
amazon_del["Order_Date"] = pd.to_datetime(amazon_del["Order_Date"]).dt.date

### **Order_Time**  
No apparent missing values.

In [181]:
amazon_del["Order_Time"] = pd.to_datetime(amazon_del["Order_Time"]).dt.time

C:\Users\Angelique\AppData\Local\Temp\ipykernel_25320\645566282.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  amazon_del["Order_Time"] = pd.to_datetime(amazon_del["Order_Time"]).dt.time


DateParseError: Unknown datetime string format, unable to parse: NaN , at position 174

However, we do have NaN starting at position 174. Why wasn't it considered as missing value?

In [ ]:
print(amazon_del.iloc[172:176])

          Order_ID  Agent_Age  Agent_Rating  Store_Latitude  Store_Longitude  \
173  wfjy432968933         28           4.8       18.530963        73.828972   
174  opdl820466364         38           4.7       12.321214        76.621094   
175  wbpn330130607         31           4.6       12.308500        76.665808   
176  jtsd466670552         25           4.8       19.055831        72.833984   

     Drop_Latitude  Drop_Longitude  Order_Date Order_Time Pickup_Time Weather  \
173      18.590963       73.888972  2022-03-03   19:00:00    19:05:00     Fog   
174      12.361214       76.661094  2022-03-30   13:10:00    13:20:00   Windy   
175      12.368500       76.725808  2022-03-19   17:55:00    18:05:00   Windy   
176      19.115831       72.893984  2022-03-15   19:50:00    20:05:00  Cloudy   

     Traffic      Vehicle            Area  Delivery_Time   Category  
173  Medium      scooter           Urban              55      Books  
174    High   motorcycle   Metropolitian             

I don't have any NaN at this position or nearby. 

In [ ]:
print(amazon_del["Order_Time"].unique())

['11:30:00' '19:45:00' '08:30:00' '18:00:00' '13:30:00' '21:20:00'
 '19:15:00' '17:25:00' '20:55:00' '21:55:00' '14:55:00' '17:30:00'
 '09:20:00' '19:50:00' '20:25:00' '20:30:00' '20:40:00' '21:15:00'
 '20:20:00' '22:30:00' '08:15:00' '19:30:00' '12:25:00' '18:35:00'
 '20:35:00' '23:20:00' '23:35:00' '22:35:00' '23:25:00' '13:35:00'
 '21:35:00' '18:55:00' '14:15:00' '11:00:00' '09:45:00' '08:40:00'
 '23:00:00' '19:10:00' '10:55:00' '21:40:00' '19:00:00' '16:45:00'
 '15:10:00' '22:45:00' '22:10:00' '20:45:00' '22:50:00' '17:55:00'
 '09:25:00' '20:15:00' '22:25:00' '22:40:00' '23:50:00' '15:25:00'
 '10:20:00' '10:40:00' '15:55:00' '20:10:00' '12:10:00' '15:30:00'
 '10:35:00' '21:10:00' '20:50:00' '12:35:00' '21:00:00' '23:40:00'
 '18:15:00' '18:20:00' '11:45:00' '12:45:00' '23:30:00' '10:50:00'
 '21:25:00' '10:10:00' '17:50:00' '22:20:00' '12:40:00' '23:55:00'
 '10:25:00' '08:45:00' '23:45:00' '19:55:00' '22:15:00' '23:10:00'
 '09:15:00' '18:25:00' '18:45:00' '16:50:00' '00:00:00' '14:20

I can see a whitespace. That is the reason why it cannot be considered as missing value.

In [ ]:
amazon_del["Order_Time"] = amazon_del["Order_Time"].str.strip()
missing_Order_Time = amazon_del[amazon_del["Order_Time"] == "NaN"]
print(missing_Order_Time.info())

<class 'pandas.core.frame.DataFrame'>
Index: 91 entries, 2286 to 43490
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order_ID         91 non-null     object 
 1   Agent_Age        91 non-null     int64  
 2   Agent_Rating     91 non-null     float64
 3   Store_Latitude   91 non-null     float64
 4   Store_Longitude  91 non-null     float64
 5   Drop_Latitude    91 non-null     float64
 6   Drop_Longitude   91 non-null     float64
 7   Order_Date       91 non-null     object 
 8   Order_Time       91 non-null     object 
 9   Pickup_Time      91 non-null     object 
 10  Weather          0 non-null      object 
 11  Traffic          91 non-null     object 
 12  Vehicle          91 non-null     object 
 13  Area             91 non-null     object 
 14  Delivery_Time    91 non-null     int64  
 15  Category         91 non-null     object 
dtypes: float64(5), int64(2), object(9)
memory usage: 12.1+ KB
None


Interestingly, all the missing values Order_Time correlate with the absence of Weather values.  
I won't remove them right away, I will tr< to understand why is that.

### **Pickup_Time**  
No apparent missing values.

In [ ]:
amazon_del["Pickup_Time"] = pd.to_datetime(amazon_del["Pickup_Time"], format="%H:%M:%S")
amazon_del["Pickup_Time"] = amazon_del["Pickup_Time"].dt.time
print(amazon_del["Pickup_Time"].head())

0    11:45:00
1    19:50:00
2    08:45:00
3    18:10:00
4    13:45:00
Name: Pickup_Time, dtype: object


### **Weather**  
Based on the info table there are 91 missing values. As observed above they all contain also missing Order_Time values.  

In [ ]:
missing_weather = amazon_del[amazon_del["Weather"].isna()]
non_missing_weather = amazon_del[~(amazon_del["Weather"].isna())]
print(missing_weather.info())

<class 'pandas.core.frame.DataFrame'>
Index: 91 entries, 2286 to 43490
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order_ID         91 non-null     object 
 1   Agent_Age        91 non-null     int64  
 2   Agent_Rating     91 non-null     float64
 3   Store_Latitude   91 non-null     float64
 4   Store_Longitude  91 non-null     float64
 5   Drop_Latitude    91 non-null     float64
 6   Drop_Longitude   91 non-null     float64
 7   Order_Date       91 non-null     object 
 8   Order_Time       0 non-null      object 
 9   Pickup_Time      91 non-null     object 
 10  Weather          0 non-null      object 
 11  Traffic          91 non-null     object 
 12  Vehicle          91 non-null     object 
 13  Area             91 non-null     object 
 14  Delivery_Time    91 non-null     int64  
 15  Category         91 non-null     object 
dtypes: float64(5), int64(2), object(9)
memory usage: 12.1+ KB
None


In [ ]:
missing_weather.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
2286,xige084493792,15,1.0,-26.891191,75.802083,26.981191,75.892083,2022-03-12,NaN,17:20:00,NaN,NaN,motorcycle,Urban,75,Home
2779,oilg311747812,15,1.0,22.750040,75.902847,22.810040,75.962847,2022-04-03,NaN,20:30:00,NaN,NaN,scooter,Metropolitian,145,Kitchen
2825,pbox816153129,15,1.0,21.149569,72.772697,21.209569,72.832697,2022-03-21,NaN,22:10:00,NaN,NaN,bicycle,Metropolitian,100,Clothing
3438,nzsa056960624,50,6.0,-12.970324,-77.645748,13.010324,77.685748,2022-03-13,NaN,12:30:00,NaN,NaN,van,Urban,125,Cosmetics
4514,eids248121351,50,6.0,-17.451976,-78.385883,17.561976,78.495883,2022-04-04,NaN,23:20:00,NaN,NaN,bicycle,Metropolitian,135,Jewelry


In [ ]:
amazon_del["Traffic"].unique()

array(['High ', 'Jam ', 'Low ', 'Medium ', 'NaN '], dtype=object)

Actually, Traffic also have NaN values due again to a white space.

In [182]:
amazon_del["Traffic"] = amazon_del["Traffic"].str.strip()
missing_weather["Traffic"] = missing_weather["Traffic"].str.strip()
print(missing_weather["Traffic"].unique())

['NaN']


In [184]:
missing_weather["Traffic"] = missing_weather["Traffic"].replace("NaN", np.nan)

C:\Users\Angelique\AppData\Local\Temp\ipykernel_25320\630223040.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  missing_weather["Traffic"] = missing_weather["Traffic"].replace("NaN", np.nan)


In [185]:
missing_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91 entries, 2286 to 43490
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order_ID         91 non-null     object 
 1   Agent_Age        91 non-null     int64  
 2   Agent_Rating     91 non-null     float64
 3   Store_Latitude   91 non-null     float64
 4   Store_Longitude  91 non-null     float64
 5   Drop_Latitude    91 non-null     float64
 6   Drop_Longitude   91 non-null     float64
 7   Order_Date       91 non-null     object 
 8   Order_Time       0 non-null      object 
 9   Pickup_Time      91 non-null     object 
 10  Weather          0 non-null      object 
 11  Traffic          0 non-null      float64
 12  Vehicle          91 non-null     object 
 13  Area             91 non-null     object 
 14  Delivery_Time    91 non-null     int64  
 15  Category         91 non-null     object 
dtypes: float64(6), int64(2), object(8)
memory usage: 12.1+ KB


We can now as well all the missing weather values also contain only Traffic null values. 

### **Vehicle**

In [187]:
amazon_del["Vehicle"].unique()

array(['motorcycle ', 'scooter ', 'van', 'bicycle '], dtype=object)

In [188]:
amazon_del["Vehicle"] = amazon_del["Vehicle"].str.strip()
print(amazon_del["Vehicle"].unique())

['motorcycle' 'scooter' 'van' 'bicycle']


In [191]:
order_vehicle = ["bicycle", "scooter", "motorcycle","van"]
amazon_del["Vehicle"] = amazon_del["Vehicle"].astype("category")
amazon_del["Vehicle"] = amazon_del["Vehicle"].cat.reorder_categories(new_categories=order_vehicle, ordered=True)

### **Area**

In [ ]:
amazon_del["Area"].unique()

array(['Urban ', 'Metropolitian ', 'Semi-Urban ', 'Other'], dtype=object)

In [189]:
amazon_del["Area"] = amazon_del["Area"].str.strip()


In [190]:
amazon_del["Area"] = amazon_del["Area"].replace("Metropolitian","Metropolitan")
order_Area = ["Other","Semi-Urban","Urban","Metropolitan"]
amazon_del["Area"] = amazon_del["Area"].astype("category")
amazon_del["Area"] = amazon_del["Area"].cat.reorder_categories(new_categories=order_Area, ordered=True)


### **Delivery_Time**  
No apparent missing values. That will be my dependent variable.

In [193]:
print(amazon_del["Delivery_Time"].describe())

count    43685.000000
mean       124.907588
std         51.924227
min         10.000000
25%         90.000000
50%        125.000000
75%        160.000000
max        270.000000
Name: Delivery_Time, dtype: float64


### **Category**  
No apparent missing values.  

In [ ]:
amazon_del["Category"].unique()

array(['Clothing', 'Electronics', 'Sports', 'Cosmetics', 'Toys', 'Snacks',
       'Shoes', 'Apparel', 'Jewelry', 'Outdoors', 'Grocery', 'Books',
       'Kitchen', 'Home', 'Pet Supplies', 'Skincare'], dtype=object)

## **Creating a column for distance between the store point and the drop point**

## **Creating a column for city and a column for country**  
I have created a module allowing to determine the city and country names using the longitude and latitude.

## **Looking more closely at the peculiarity of the missing weather values**  
Can I figure why I have three features missing (wheather, Order_Time and Traffic)?